In [9]:
from keras.models import Sequential
from keras.layers import Lambda, Dense, GlobalAveragePooling2D
import tensorflow as tf
import keras.applications.mobilenet_v2 as mobilenetv2
from dotenv import load_dotenv
import os
import cv2
import numpy as np

load_dotenv()

True

In [3]:
# Image dimensions
IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS = 224, 224, 3

# Original Category mappings (12 categories)
categories = {0: 'paper', 1: 'cardboard', 2: 'plastic', 3: 'metal', 4: 'trash', 
              5: 'battery', 6: 'shoes', 7: 'clothes', 8: 'green-glass', 
              9: 'brown-glass', 10: 'white-glass', 11: 'biological'}

In [4]:
# The model's internal category indexing not align with the Original Category mappings
# Reverse engineer to find the correct classification label (Plastic) - using train set plastic image
def map_to_plastic_or_nonplastic(predicted_class):
    # 3 is Plastic
    if predicted_class == 3:
        return "Plastic"
    else:
        return "Non-Plastic"

In [5]:
pretrained_model_weight = os.getenv("PRETRAINED_MODEL")

In [6]:
import h5py

# Open the .h5 file
with h5py.File(pretrained_model_weight, 'r') as f:
    # Print all layer names and their weights
    for layer_name in f.keys():
        print(f"Layer name: {layer_name}")
        for weight_name in f[layer_name].keys():
            print(f"    Weight name: {weight_name}, shape: {f[layer_name][weight_name]}")


Layer name: dense
    Weight name: dense, shape: <HDF5 group "/dense/dense" (2 members)>
Layer name: global_average_pooling2d
Layer name: lambda
Layer name: mobilenetv2_1.00_224
    Weight name: Conv1, shape: <HDF5 group "/mobilenetv2_1.00_224/Conv1" (1 members)>
    Weight name: Conv_1, shape: <HDF5 group "/mobilenetv2_1.00_224/Conv_1" (1 members)>
    Weight name: Conv_1_bn, shape: <HDF5 group "/mobilenetv2_1.00_224/Conv_1_bn" (4 members)>
    Weight name: block_10_depthwise, shape: <HDF5 group "/mobilenetv2_1.00_224/block_10_depthwise" (1 members)>
    Weight name: block_10_depthwise_BN, shape: <HDF5 group "/mobilenetv2_1.00_224/block_10_depthwise_BN" (4 members)>
    Weight name: block_10_expand, shape: <HDF5 group "/mobilenetv2_1.00_224/block_10_expand" (1 members)>
    Weight name: block_10_expand_BN, shape: <HDF5 group "/mobilenetv2_1.00_224/block_10_expand_BN" (4 members)>
    Weight name: block_10_project, shape: <HDF5 group "/mobilenetv2_1.00_224/block_10_project" (1 members)

### Transfer Learning – MobileNetV2 weights are frozen, and only the classification head is trained for waste classification.

MobileNetV2 weights - https://www.kaggle.com/code/alexfordna/garbage-classification-mobilenetv2-92-accuracy/input?select=garbage_classification

In [7]:
# Load MobileNetV2 base model without the top layer
mobilenetv2_layer = mobilenetv2.MobileNetV2(include_top=False, 
                                            input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS),
                                            weights='mobilenet-v2-keras-weights/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224_no_top.h5')

mobilenetv2_layer.trainable = False


# Create the model architecture
model = Sequential()
# Input layer
model.add(tf.keras.Input(shape=(IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS)))

# Lambda preprocessing layer
def mobilenetv2_preprocessing(img):
    return mobilenetv2.preprocess_input(img)
model.add(Lambda(mobilenetv2_preprocessing))

model.add(mobilenetv2_layer)

# Add the GlobalAveragePooling2D layer
model.add(GlobalAveragePooling2D())

# Add the Dense layer for classification
model.add(Dense(12, activation='softmax'))

# Load the weights from your .h5 file
model.load_weights(pretrained_model_weight)

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

# Print the model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lambda (Lambda)             (None, 224, 224, 3)       0         
                                                                 
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 12)                15372     
                                                                 
Total params: 2,273,356
Trainable params: 15,372
Non-trainable params: 2,257,984
_________________________________________________________________


In [8]:
if hasattr(model, 'class_indices'):
    print("Class indices found:", model.class_indices)
else:
    print("No class indices found in the model.")

No class indices found in the model.


In [10]:
def preprocess_image(image_path):
    image = cv2.imread(image_path)
    
    # Resize the image to 224x224
    image_resized = cv2.resize(image, (224, 224))
    
    # Convert image to float32 and normalize using MobileNetV2's preprocess_input
    image_normalized = mobilenetv2.preprocess_input(image_resized.astype(np.float32))
    
    # Add a batch dimension (the model expects input shape: (batch_size, 224, 224, 3))
    image_batch = np.expand_dims(image_normalized, axis=0)
    
    return image_batch

In [11]:
def classify_image(model, image_path):
    # Preprocess the image
    image_batch = preprocess_image(image_path)
    
    # Perform inference
    predictions = model.predict(image_batch)
    
    # Get the predicted class (argmax - the index of the highest probability)
    predicted_class = np.argmax(predictions, axis=1)[0]
    
    # Map the predicted class to plastic or non-plastic before returning
    return map_to_plastic_or_nonplastic(predicted_class)

In [12]:
image_path = 'sample_data/plastic146_in_dataset.jpg'

result = classify_image(model, image_path)

print(f"The object is classified as: {result}")

1/1 [==============================] - 1s 640ms/step
The object is classified as: Plastic


convert to tflite model

In [9]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\User\AppData\Local\Temp\tmpahgd3g4u\assets


INFO:tensorflow:Assets written to: C:\Users\User\AppData\Local\Temp\tmpahgd3g4u\assets


Saved artifact at 'C:\Users\User\AppData\Local\Temp\tmpahgd3g4u'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor_154')
Output Type:
  TensorSpec(shape=(None, 12), dtype=tf.float32, name=None)
Captures:
  1675782403472: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1675782405008: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1675782403856: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1675782404624: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1675782405584: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1675782404432: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1675782405776: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1675783110736: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1675782402320: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1675782405392: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1

In [15]:
tflite_model_save_path = os.getenv("TF_LITE_MODEL")

In [10]:
with open(tflite_model_save_path, 'wb') as f:
    f.write(tflite_model)

print("Model successfully converted to TensorFlow Lite and saved!")

Model successfully converted to TensorFlow Lite!
